# Part 4, Model 2A NN: Glaucoma Detection using a neural network on the fundus images of the eye + selected patient details.

In [ ]:
%pip install scikit-learn
%pip install seaborn

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v3 import preprocess_input 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler  # Fix import statement
# to prevent unnecessary warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# TensorFlow and tf.keras
import tensorflow as tf

from pathlib import Path

#import useful module for keras library
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

# get modules from sklearn library
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

#import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pandas as pd

In [ ]:
tf.__version__

In [ ]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('metadata - standardized.csv')

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
# know column type
data['fundus_oc_seg'].dtype

In [ ]:
data.info()

In [ ]:
print(data['fundus'][12044])
print(data['fundus_oc_seg'][12044])
print(data['fundus_od_seg'][12044])

In [ ]:
def viewFundus(image_path):

    image_path = 'full-fundus' + image_path

    print(image_path)

    # Load the image
    image = cv2.imread(image_path)

    # Convert the image from BGR to RGB color space
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image
    plt.imshow(image_rgb)
    plt.axis('on')  # Hide the axis to only show the image
    plt.show()

viewFundus(data['fundus'][12044])

In [ ]:
data.head()

In [ ]:
data['eye'] = data['eye'].replace({'OD': 0, 'OS': 1})

data[['eye']].value_counts()

In [ ]:
data.info()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Assuming 'data' is your original DataFrame

# Separate the data into numerical and non-numerical subsets
numerical_data = data.select_dtypes(include=[np.number])  # only numerical features

# Check and fill columns with all NaN values
for col in numerical_data.columns:
    if numerical_data[col].isna().all():
        numerical_data[col].fillna(0, inplace=True)  # or use another strategy

# Check the number of columns before imputation
print("Number of columns before imputation:", numerical_data.shape[1])

# Initialize the IterativeImputer with a RandomForestRegressor
imp = IterativeImputer(RandomForestRegressor(), max_iter=10, random_state=0)

# Perform the imputation - fit and transform the numerical data
numerical_data_imputed = imp.fit_transform(numerical_data)

# Convert the imputed numerical data back to a pandas DataFrame
# Debugging: Check the shape of the imputed data
print("Shape of imputed data:", numerical_data_imputed.shape)

# Create the DataFrame using the original column names
# Here we're assuming that the columns should match the original
# If not, adjust the columns parameter accordingly
numerical_data_imputed = pd.DataFrame(numerical_data_imputed, columns=numerical_data.columns, index=numerical_data.index)

# Combine the imputed numerical data back with the non-numerical data
non_numerical_data = data.select_dtypes(exclude=[np.number])  # reselect to include changes
data_imputed = pd.concat([numerical_data_imputed, non_numerical_data], axis=1)

# Ensure the order of rows remains the same
data_imputed = data_imputed.loc[data.index]

In [ ]:
print(data_imputed.describe())

In [ ]:
data_imputed[["eye"]].value_counts()

In [ ]:
data_imputed['eye'] = data_imputed['eye'].round().astype(int)

In [ ]:
data_imputed[["eye"]].value_counts()

In [ ]:
data = data_imputed

data

In [ ]:
#removing the patients with no fundus classification and with fundus classification -1 instead of 0,1
data = data[
    
    (data['fundus'].notnull()) & (data['types'] != -1)
    
]

fundus_subset = data[['types', 'fundus', 'names']]

fundus_subset['combined'] = fundus_subset['names'] + '.png'

fundus_subset['fundus'] = fundus_subset['fundus'].astype(str)

fundus_subset['types'] = fundus_subset['types'].astype(str)

fundus_subset.info()


In [ ]:
fundus_subset['types'].value_counts()

In [ ]:
fundus_subset

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# Calculate the number of samples for each class
glaucomaCount = len(fundus_subset[fundus_subset['types'] == '1.0'])
healthyCount = len(fundus_subset[fundus_subset['types'] == '0.0'])

print("this is the glaucoma count", glaucomaCount)
print("this is the healthy eyes count", healthyCount)

# Set the percentages for the test and validation splits
test_percentage = 0.20  # 20% of the data for testing
val_percentage_from_train = 0.15  # 25% of the remaining data after test split for validation

# Assuming fundus_subset is your DataFrame and 'types' is the column with labels

# Set the random seed for reproducibility
random_state = 1

# Create StratifiedShuffleSplit instance for test set with percentage
stratified_split_test = StratifiedShuffleSplit(n_splits=1, test_size=test_percentage, random_state=random_state)

for train_index, test_index in stratified_split_test.split(fundus_subset, fundus_subset['types']):
    train_set_temp = fundus_subset.iloc[train_index]
    test = fundus_subset.iloc[test_index]

# Note: Adjusted validation split to calculate percentage from the remaining train set
# Create StratifiedShuffleSplit instance for validation set with percentage
stratified_split_val = StratifiedShuffleSplit(n_splits=1, test_size=val_percentage_from_train, random_state=random_state)

for train_index, val_index in stratified_split_val.split(train_set_temp, train_set_temp['types']):
    train = train_set_temp.iloc[train_index]
    val = train_set_temp.iloc[val_index]

# Print the counts for each set and total
print("\nTotal fundus images to be used:", len(fundus_subset))
print("\nTrain set size:", len(train))
print("Validation set size:", len(val))
print("Test set size:", len(test))

# You can access the features and labels as needed
train_features = train.drop('types', axis=1)  # assuming 'types' is the label column
train_labels = train['types']
val_features = val.drop('types', axis=1)
val_labels = val['types']
test_features = test.drop('types', axis=1)
test_labels = test['types']

In [ ]:
print("Train Size: ", len(train))
print("Test Size: ", len(test))

In [ ]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers.experimental.preprocessing import Rescaling
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v3 import preprocess_input 


# Set target size and batch size for data generator
target = 224
batchSize = 8

# Set the class mode to 'binary' for training data generator
classes = 'binary'

with tf.device('/GPU:0'):


    # Create the training data generator
    trainDataGen = ImageDataGenerator(preprocessing_function=preprocess_input)
    trainGen = trainDataGen.flow_from_dataframe(dataframe=train, 
                                                directory='full-fundus/full-fundus', 
                                                class_mode = classes,
                                                batch_size = batchSize, 
                                                shuffle=True, 
                                                x_col="combined", 
                                                y_col="types", 
                                                validate_filenames=True, 
                                                target_size=(target, target), 
                                                color_mode='rgb')


    # Create the validation data generator
    valDataGen = ImageDataGenerator(preprocessing_function=preprocess_input)
    valGen = valDataGen.flow_from_dataframe(dataframe=val,
                                            directory='full-fundus/full-fundus',   
                                            batch_size = batchSize, 
                                            class_mode = classes,
                                            shuffle=False, 
                                            x_col="combined", 
                                            y_col="types", 
                                            validate_filenames=True, 
                                            target_size=(target, target), 
                                            color_mode='rgb')


    # Create the testing data generator
    testDataGen = ImageDataGenerator(preprocessing_function=preprocess_input)
    testGen = testDataGen.flow_from_dataframe(dataframe=test,
                                            directory='full-fundus/full-fundus',   
                                            batch_size = batchSize, 
                                            class_mode = classes,
                                            shuffle=False, 
                                            x_col="combined", 
                                            y_col="types", 
                                            validate_filenames=True, 
                                            target_size=(target, target), 
                                            color_mode='rgb')

In [ ]:
# Select the non-image features from your dataframe
non_image_features = data[['gender', 'age', 'eye', 'sbp', 'dbp', 'hr', 'iop', 'vcdr', 'cdr_avg']]